# Annotating agency for Lev 17-26

#### Annotated files

* Original annotation: [Lev17-26.Volition_Affectedness_all.csv](datasets/Lev17-26.Volition_Affectedness_all.csv)
* First check: Checking cases that are commented ore question-marked: [Lev17-26.Volition_Affectedness_all_cor_1.csv](datasets/Lev17-26.Volition_Affectedness_all_cor_1.csv)
* Second check: Checking verbs for which Actor and Undergoer(s) are annotated differently: [Lev17-26.Volition_Affectedness_all_cor_2.csv](datasets/Lev17-26.Volition_Affectedness_all_cor_2.csv)
* Third check: Manual inspection of all complement phrases in Lev 16-27: [Lev17-26.Volition_Affectedness_all_cor_3.csv](datasets/Lev17-26.Volition_Affectedness_all_cor_3.csv)

In [1]:
#Dataset path
PATH = 'datasets/'

In [3]:
from IPython.display import clear_output

import collections
from operator import itemgetter
import pandas as pd

from tf.app import use

In [4]:
A = use('bhsa', hoist=globals(), mod='etcbc/heads/tf')

TF app is up-to-date.
Using annotation/app-bhsa commit 5fdf1778d51d938bfe80b37b415e36618e50190c (=latest)
  in C:\Users\Ejer/text-fabric-data/__apps__/bhsa.
Using etcbc/bhsa/tf - c r1.4 in C:\Users\Ejer/text-fabric-data
Using etcbc/phono/tf - c r1.1 in C:\Users\Ejer/text-fabric-data
Using etcbc/parallels/tf - c r1.1 in C:\Users\Ejer/text-fabric-data
Using etcbc/heads/tf - c rv.1.11 in C:\Users\Ejer/text-fabric-data


In [3]:
all_verbs = '''
book book=Leviticus
 chapter chapter=17|18|19|20|21|22|23|24|25|26
  clause
   phrase function=Pred|PreO|PreS|PreC|PtcO
     word pdp=verb
'''

all_verbs = A.search(all_verbs)

  1.28s 936 results


In [4]:
total_clauses = [r[2] for r in all_verbs]
len(set(total_clauses))

936

## Annotating Volition and Affectedness

#### Initial functions and variables

In [5]:
anno = 'y n ?'.split()

STOP = 'stop'
error_message = "Input is invalid"

In [6]:
import subprocess

def LogosLink(node, bible='bhssesb'):
    bo, ch, ve = T.sectionFromNode(node)
    logos_link = f'logosres:{bible};ref=BibleBHS.{bo}{ch}.{ve}'
    
    #Open Logos
    path='C:/Users/Ejer/AppData/Local/Logos/'
    p = subprocess.Popen([f'{path}Logos.exe', logos_link])

#LogosLink(250000)

In [7]:
def input_loop(question, right_answer):
    '''
    input_loop is used when the user is required to make an input. Question has to be a string,
    while right_answer is a list of accepted answer(s).
    '''
    while True:
        test = input(f'{question} ({",".join(right_answer)})') #The user is required to respond to message.
        if test in right_answer:
            return test
        elif test in [STOP]: #Input is accepted
            return test
        else: #Input is wrong and the user has respond again.
            print(error_message)

def get_head(ph):
    nom_head = E.nhead.t(ph)
    if len(nom_head) > 0:
        return nom_head[0]
    else:
        return E.head.t(ph)[0]

In [23]:
def user_input(clauses):
    
    annotations = {}
    
    for clause_node in clauses:
    
        clear_output()
        
        Act = ''
        Act_Vol = ''
        Act_Aff = ''
        Und1 = ''
        Und1_Vol = ''
        Und1_Aff = ''
        Und2 = ''
        Und2_Vol = ''
        Und2_Aff = ''

        #Defining possible Actor and Undergoer syntactic functions
        Actor = ['Subj','Pred','PreS','PreO','PreC','PtcO','Rela']
        Undergoer1 = ['Objc','PreO','PtcO','Cmpl','Rela']
        Undergoer2 = ['Objc','PreO','PtcO','Cmpl','Rela']

        #If passive voice, Actor and Undergoer are inverted
        for ph in L.d(clause_node, 'phrase'):
            if F.function.v(ph) in ['Pred','PreS','PreO','PreC','PtcO']:
                for w in L.d(ph, 'word'):
                    if F.pdp.v(w) == 'verb' and F.vs.v(w) in ['nif','pual','hof']:
                        Actor = ['Objc','PreO','PtcO','Cmpl','Rela']
                        Undergoer1 = ['Subj','Pred','PreS','PreO','PreC','PtcO','Rela']   

        test = ''

        LogosLink(clause_node)

        for ph in L.d(clause_node, 'phrase'):

            head = get_head(ph) #Find nominal or prepositional head.

            #Annotate Actor
            if F.function.v(ph) in Actor:

                #Visualize clause and highlight headword
                A.pretty(clause_node, highlights = {head: 'gold'})

                #1. test
                test = input_loop("Actor", anno)

                if test == 'y':
                    Act = ph

                    test = input_loop("Volition", anno)

                    if test in anno:
                        Act_Vol = test

                        test = input_loop("Affectedness", anno)
                        if test in anno:
                            Act_Aff = test
                            break

                        elif test == STOP: break

                    elif test == STOP: break

                elif test == '?':
                    Act = '?'
                    break

                elif test == 'n': continue

                elif test == STOP: break

        if test != STOP:
            for ph in L.d(clause_node, 'phrase'):

                head = get_head(ph) #Find nominal or prepositional head.

                #Annotate Undergoer1
                if test != STOP and F.function.v(ph) in Undergoer1:

                    #Visualize clause and highlight headword
                    A.pretty(clause_node, highlights = {head: 'salmon'})

                    #1. test
                    test = input_loop("Undergoer 1", ['y','n','?'])

                    if test == 'y':
                        Und1 = ph

                        test = input_loop("Volition", anno)

                        if test in anno:
                            Und1_Vol = test

                            test = input_loop("Affectedness", anno)
                            if test in anno:
                                Und1_Aff = test
                                break

                            elif test == STOP: break

                        elif test == STOP: break

                    elif test == '?':
                        Und1 = '?'
                        break

                    elif test == 'n': continue

                    elif test == STOP: break

        if test != STOP:
            for ph in L.d(clause_node, 'phrase'):

                head = get_head(ph) #Find nominal or prepositional head.

                #Annotate Undergoer2
                if test != STOP and F.function.v(ph) in Undergoer1:

                    #Visualize clause and highlight headword
                    A.pretty(clause_node, highlights = {head: 'salmon'})

                    #1. test
                    test = input_loop("Undergoer 2", ['y','n','?'])

                    if test == 'y':
                        Und2 = ph

                        test = input_loop("Volition", anno)

                        if test in anno:
                            Und2_Vol = test

                            test = input_loop("Affectedness", anno)
                            if test in anno:
                                Und2_Aff = test
                                break

                            elif test == STOP: break

                        elif test == STOP: break

                    elif test == '?':
                        Und1 = '?'
                        break

                    elif test == 'n': continue

                    elif test == STOP: break

        if test != STOP:
            
            comment = input('Make comment')
        
        if test == STOP:
            return annotations
        else:
            annotations[clause_node] = [Act, Act_Vol, Act_Aff, Und1, Und1_Vol, Und1_Aff, Und2, Und2_Vol, Und2_Aff, comment]
            
    return annotations

In [32]:
datasets = ['Lev17-26.Volition_Affectedness_chap18-20.csv',
            'Lev17-26.Volition_Affectedness_chap21-23.csv','Lev17-26.Volition_Affectedness_chap24-25.csv',
            'Lev17-26.Volition_Affectedness_chap26a.csv','Lev17-26.Volition_Affectedness_chap26b.csv']

prev_runs = [run4]

def Clauses(total_clauses, datasets, prev_runs):
        
    annotated_clauses = []
    remaining_clauses = []
    
    ##1. Check annotated datasets
    for csv in datasets:
        data = pd.read_csv(csv)
        annotated_clauses += list(data.iloc[:,0])
        
    ##2. Check previous runs (dictionaries with annotations)
    for r in prev_runs:
        for cl in r:
            annotated_clauses.append(cl)
            
    ##3. Determine the remaining clauses
    for cl in total_clauses:
        if not cl in annotated_clauses:
            remaining_clauses.append(cl)
    
    return remaining_clauses
        
remaining_clauses = Clauses(total_clauses, datasets, prev_runs)

In [33]:
len(remaining_clauses)

0

In [31]:
run4 = user_input(remaining_clauses)

Actor (y,n,?)y
Volition (y,n,?)n
Affectedness (y,n,?)y


Undergoer 1 (y,n,?)y
Volition (y,n,?)n
Affectedness (y,n,?)n


Undergoer 2 (y,n,?)n
Make commentUndergoer1 is abstract so therefore not affected. The Actor is affected because he is in effect accountable for sin.


#### Merge outputs

In [34]:
outputs = [run4]

new_dict = {}
for r in outputs:
    for lex in r:
        new_dict[lex] = r[lex]
        
df = pd.DataFrame(new_dict).T
df.head()

,0,1,2,3,4,5,6,7,8,9
439650,688348,y,n,688349,n,n,,,,
439651,688350,y,n,,,,,,,
439652,688351,y,n,688352,n,n,,,,
439653,688354,y,n,688355,n,n,,,,
439655,688360,y,n,688358,n,n,,,,


In [35]:
df.to_csv(f'{PATH}Lev17-26.Volition_Affectedness_chap17.csv')

#### Merge exports

Having completed the annotation, all annotated files can now be combined into one CSV-file:

In [7]:
datasets = ['Lev17-26.Volition_Affectedness_chap17.csv','Lev17-26.Volition_Affectedness_chap18-20.csv',
            'Lev17-26.Volition_Affectedness_chap21-23.csv','Lev17-26.Volition_Affectedness_chap24-25.csv',
            'Lev17-26.Volition_Affectedness_chap26a.csv','Lev17-26.Volition_Affectedness_chap26b.csv']

new_df = pd.DataFrame()

for csv in datasets:
    df = pd.read_csv(csv)
    new_df = new_df.append(df)
    
len(new_df)

936

In [48]:
new_df.columns = ['clause','Act_phr','Act_vol','Act_aff','Und1_phr','Und1_vol','Und1_aff','Und2_phr','Und2_vol','Und2_aff',
                 'comment']

new_df.head()

,clause,Act_phr,Act_vol,Act_aff,Und1_phr,Und1_vol,Und1_aff,Und2_phr,Und2_vol,Und2_aff,comment
0,439650,688348,y,n,688349,n,n,NaN,NaN,NaN,NaN
1,439651,688350,y,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,439652,688351,y,n,688352,n,n,NaN,NaN,NaN,NaN
3,439653,688354,y,n,688355,n,n,NaN,NaN,NaN,NaN
4,439655,688360,y,n,688358,n,n,NaN,NaN,NaN,NaN


In [49]:
new_df.to_csv('Lev17-26.Volition_Affectedness_all.csv')

### Reviewing annotation

In [8]:
#A dictionary of columns to be imported as integers.
int_cols = {col:'Int64' for col in ['clause','Act_phr','Und1_phr','Und2_phr']}

data = pd.read_csv(f'{PATH}Lev17-26.Volition_Affectedness_all.csv', dtype=int_cols)

data.head()

,Unnamed: 0,clause,Act_phr,Act_vol,Act_aff,Und1_phr,Und1_vol,Und1_aff,Und2_phr,Und2_vol,Und2_aff,comment
0,0,439650,688348,y,n,688349,n,n,NaN,NaN,NaN,NaN
1,1,439651,688350,y,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,439652,688351,y,n,688352,n,n,NaN,NaN,NaN,NaN
3,3,439653,688354,y,n,688355,n,n,NaN,NaN,NaN,NaN
4,4,439655,688360,y,n,688358,n,n,NaN,NaN,NaN,NaN


In [6]:
lexemes = []

for cl in list(data.clause):
    for w in L.d(cl, 'word'):
        if F.pdp.v(w) == 'verb':
            lexemes.append(F.lex.v(w))
            
len(lexemes)

#Insert as new column
data.insert(2, "lex", lexemes)

In [7]:
data.head()

,Unnamed: 0,clause,lex,Act_phr,Act_vol,Act_aff,Und1_phr,Und1_vol,Und1_aff,Und2_phr,Und2_vol,Und2_aff,comment
0,0,439650,DBR[,688348,y,n,688349,n,n,NaN,NaN,NaN,NaN
1,1,439651,>MR[,688350,y,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,439652,DBR[,688351,y,n,688352,n,n,NaN,NaN,NaN,NaN
3,3,439653,>MR[,688354,y,n,688355,n,n,NaN,NaN,NaN,NaN
4,4,439655,YWH[,688360,y,n,688358,n,n,NaN,NaN,NaN,NaN


We now subset all rows with a question mark and/or a comment for further inspection:

In [8]:
df = data[(~data.comment.isna()) | (data.Act_vol.isin(['?'])) | (data.Act_aff.isin(['?'])) | (data.Und1_vol.isin(['?']))
    | (data.Und1_aff.isin(['?'])) | (data.Und2_vol.isin(['?'])) | (data.Und2_aff.isin(['?']))]

print(f'Number of cases: {len(df)}')

Number of cases: 271


Which verbs are involved?

In [9]:
lexemes = collections.Counter(list(df.lex))

In [12]:
lexemes.most_common(10)

[('HJH[', 27),
 ('NTN[', 16),
 ('LQX[', 13),
 ('VM>[', 12),
 ('<FH[', 11),
 ('NF>[', 9),
 ('BW>[', 6),
 ('>KL[', 6),
 ('CKB[', 6),
 ('QDC[', 6)]

We will walk through the indiviual verbs as groups although the participants will not necessarily be annotated similarly in all cases. However, some of the ambiguities pertaining to the verbs may be relevant for more cases.

To begin with, we skip the "dummy-verb" HJH.

In [13]:
#We create a list of verbs sorted according to frequency. The most common to be reviewed first:
lex = list(collections.OrderedDict(sorted(lexemes.items(), key=itemgetter(1), reverse=True)))

Only the most important discussions will be provided here. Many choices are explained in the comments in the dataset.

In [267]:
def show(lex, data=df, highlight=str):
    
    subset = data[data.lex == lex]
    clauses = list(subset.clause)
    
    colormap = {}
    if highlight:
        colormap = {c:'gold' for c in list(subset[highlight])}

    for cl in clauses:
        display(subset[subset.clause == cl])
        A.pretty(cl, highlights=colormap)

#show(lex[1])

#### LQX "take"

General notes:
1. It is difficult to know whether the object (a woman) is voluntarily "taken", that is, married to a man, or whether she is depicted as indifferent or even agaist her will. The most natural answer would be that the will of the woman is not within the horizon of the text. At least some of the rules seem to express a protection of the woman which suggests that she might not be willing.
2. "To take" is a transfer where the instigator is also the recipient. For that reason, at a number of occasions the Actor is also marked as affected.

In [460]:
#show(lex[2])

#### VM> "be unclean"

One of the main challenges of annotating VM> is the frequent complement "with it" (ב) or "to it" (ל), both indicating the source of pollution, or perhaps the instrument for contamination. The usual annotation has been a question mark because it is difficult to assess the specific meaning of this complement. 

In [462]:
#show(lex[3])

#### <FH "make"

The verb frequently occurs with the complement "for someone" (ל) which indicates a benefactor. The benefactor is not involved in the event but has an interest in it (in so far it is human) and is indirectly affected by it.

In [463]:
#show(lex[4])

#### NF> "carry"

The verb occurs most often with "sin": "carry his sin", in other words, "he is responsible". I interpret the verb as in non-volitional event because the participant does not want to carry this load. The Undergoer, "the sin" is an abstract concept and cannot be affected.

In [80]:
#show(lex[5])

#### >KL "eat"

The verb often occurs with indefinite objects which suggest that the object may not in fact be affected by the event, because there is no real-world meal intended. This phenomenon pertains to other verbs as well, of course.

In [471]:
#show(lex[7])

#### CKB "lie down"

The verb normally means to lie down but here it exclusively means "to have sex with". The question is of course wether the other participant (the Undergoer) is willingly engaging or against his/her will. The answer is not provided in the text so all cases have been question marked for Undergoer volition.

In [53]:
#show(lex[8])

In the case of adultery expressed by the verb N>P "commit adultery", it seems to be intentional for both parties. Here, the Undergoer is probably volitionalּ. Both parties are involved and affected.

In [55]:
#show(lex[34])

See also RB<= "lie down":

In [51]:
#show(lex[49])

#### ZNH "fornicate"

One of the oblique objects often occuring with ZNH refers to Molok, apparently a god. However, should it be represented as a human being? Perhaps it should in this context because a very concrete, animate verb is used for the relationship between Molok and his worshippers, that of "fornication". The object is normally affected and probably also volitional.

In [478]:
#show(lex[10])

See also ZBX[ "slaughter"

In [510]:
#show(lex[27])

#### NFG "overtake"

In a few constructions the verb is used to express "to afford". In these cases the subject "the hand" is taken as referring to the human being and therefore volitional and affected.

In [483]:
#show(lex[14])

See also MY> "find":

In [41]:
#show(lex[42])

#### Harvest terms

A number of terms are used for harvest. One of the basic meanings seems to be "gathering" of the crops because the addressees of the text are explicitly told to "leave" some of the crops to the poor. The issue at stake is therefore not the cutting of the crops itself but the gathering.

Therefore, I interpret the verbs primarily as dealing with a transfer where the instigator is also the recipient.

In [504]:
#show(lex[25])

In [513]:
#show(lex[29])

In [36]:
#show(lex[39])

In [59]:
#show(lex[52])

#### RYH "pay off"

The verb apparently has two meanings in Lev 26. It seems to mean "enjoy" (in the context of sabbaths) but also to mean "pay off" (in the context of sin). The former case is a fealing and the Actor is then affected. In the latter case, what may be meant by the verb is some kind of "penitence" which involves a cognitive process. Therefore, also in this case I regard the Actor is affected. The Undergoers of both meanings are not affected because they are abstract concepts.

In [508]:
#show(lex[26])

#### XLL "defile"

To defile can both be intentional and unintentional like any other verbs. In practice, however, it is difficult to distinguish unless the text specifies that the defilement happened by accident. In fact, the text seems to accuse the participants for willingly defiling the sanctuary and themselves.

In [21]:
#show(lex[33])

#### JY> "go out"

The verb often occurs with the complement of deprivation "from somewhere/someone". Normally that participant is non-volitionally deprived of the Actor and therefore affected. The annotation will resemble the "malefactive".

In [31]:
#show(lex[37])

#### YWH "command"

The Undergoer of "command" is affected because it senses the utterance and is cognitively activated. The Undergoer is probably also volitional in so far as the command itself is not maleficient to the participant.

In [33]:
#show(lex[38])

#### MKR "sell"

"To sell" involves a seller and a buyer. Both are affected because the seller looses his possession while the buyer comes into possession. In Hebrew both aspects can be emphasized in the text but need not be.

In [39]:
#show(lex[41])

#### GNB "steal"

Steal involves both a recipient and a source who is deprived of an object. The focus seems to be on the one being deprived and not on the party who benefits from the event.

In [61]:
#show(lex[53])

See also GZL "tear away":

In [67]:
#show(lex[56])

#### <MD "stand"

In Lev 19:16 the verb is used in peculiar construction: "You shall not stand upon the blood of your fellow". The clause has been translated differently, such as "you shall not profit by the blood of your neighbor". The Actor is apparently affected in that he gains from the event. The Undergoer is negatively affected.

In [71]:
#show(lex[59])

#### MVH "totter"

The verb comes with a complement "with (you)" that may be interpreted as the orientation of the event: "and his hand is waving toward you/in your direction". More likely, however, the complement describes the location, similar to "in your midst". In that case the Undergoer described by the complement phrase is not involved in the event nor affected.

In [122]:
#show(lex[91])

A similar example is found with MWK "grow poor":

In [123]:
#show(lex[93])

### Next steps

1. Cross-check all verbs. If the Actor and Undergoer(s) are annotated differently, then investigate
2. Manually check all complement phrases (not marked for Volition and Affectedness)

### Cross-checking all verbs

In this step of validation of all verbs are investigated as to the annotation of each participant (Actor, Undergoer1 and Undergoer2). That participants may be annotated differently is not a in itself a problem because the role of the participant is a relational feature and not predicated by the verb. Nevertheless, the participants of a particular verb may often share certain features and therefore be annotated similarly.

By undertaking this kind of validation, we explore the dataset from a different angle which may avoid biases coming from investigation the dataset from A to Z. 

We first import the corrected dataset and add a new column with lexemes:

In [9]:
#A dictionary of columns to be imported as integers.
int_cols = {col:'Int64' for col in ['clause','Act_phr','Und1_phr','Und2_phr']}

data = pd.read_csv(f'{PATH}Lev17-26.Volition_Affectedness_all_cor.csv', dtype=int_cols)

In [148]:
lexemes = []

for cl in list(data.clause):
    for w in L.d(cl, 'word'):
        if F.pdp.v(w) == 'verb':
            lexemes.append(F.lex.v(w))
            
len(lexemes)

#Insert as new column
data.insert(2, "lex", lexemes)

In [149]:
data.head()

,Unnamed: 0,clause,lex,Act_phr,Act_vol,Act_aff,Und1_phr,Und1_vol,Und1_aff,Und2_phr,Und2_vol,Und2_aff,comment
0,0,439650,DBR[,688348,y,n,688349,n,n,NaN,NaN,NaN,NaN
1,1,439651,>MR[,688350,y,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,439652,DBR[,688351,y,n,688352,n,n,NaN,NaN,NaN,NaN
3,3,439653,>MR[,688354,y,n,688355,n,n,NaN,NaN,NaN,NaN
4,4,439655,YWH[,688360,y,n,688358,n,n,NaN,NaN,NaN,NaN


The resulting dataset is exported:

In [207]:
data.to_csv(f'{PATH}Lev17-26.Volition_Affectedness_all_cor_1.csv')

We are now ready to evaluate the participants of each lexeme. The procedure is simply to create subsets for each participant in the dataset and to explore whether they contain more than one annotation (excluding empty cells):

In [176]:
def evaluate(role, df=data):
    mismatches = set()
    
    for lex in list(df.lex):
        subset = df[df.lex == lex]
        
        aspects = ['vol','aff']
        
        for asp in aspects:
            anno = subset[subset[f'{role}_{asp}'].notna()][f'{role}_{asp}']
            anno = set(anno)
        
            if len(anno) > 1:
                mismatches.add(lex)
                
    return list(mismatches)

#### Actor

In [184]:
Actor = evaluate('Act')
len(Actor)

47

47 verbs are annotated differently with respect to the Actor role. We will explore them one by one and discuss the most important implications here:

##### Harvest verbs

All harvest verbs are treated as expressing "gathering" or "collecting", therefore a transfer where the Actor is also the recipient (unless otherwise stated in the clause):

In [192]:
#show(Actor[4], data)

In [193]:
#show(Actor[5], data)

In [243]:
#show(Actor[44], data)

##### *Qal* - *Hiphil* alternations

Many differences in the affectedness of the Actor are uncontroversial and stem from the alternation between *Qal* and *Hiphil* where the *Hiphil* expresses a causative situation where the Undergoer is undergoing the event in contrast to *Qal* where the Actor is undergoing the event, e.g. QRB "approach":

In [201]:
#show(Actor[6], data)

##### Human and nonhuman participants

One major deviation is caused by the fact that many verbs occur both with human (normally volitional) and nonhuman (nonvolitional) participants, e.g.:

In [219]:
#show(Actor[21], data)

#### Undergoer 1

In [269]:
Und1 = evaluate('Und1')
print(f'Number of verbs to be evaluated: {len(Und1)}')

Number of verbs to be evaluated: 48


##### VM> "be unclean"

The verb is usually found in combination with a complement phrase denoting either source or instrument. The complement phrase is formed either by ל "to/by" or ב "with". The prepositions can have meanings, though. Therefore, the usual annotation has been to treat the Undergoers as neutral, that is, not affected and not instigating. It is not obvious to see a woman or a body as an Instrument, because who is handling the "instrument". It is better to perceive the complement as denoting a source of contamination, the source itself not being affected because it is already unclean. 

In [289]:
#show(Und1[15], data, highlight='Und1_phr')

##### >KL "eat"

The Undergoer of "eat" is usually the thing consumed. Normally, one can expect the Undergoer to be affected because it is eaten. However, many times in this text the Undergoer is not specific as to an amount of food. On the other hand, it is difficult to distinguish specific and unspecific entities because it is a juridical text dealing with hypothetical cases that are assumed to take place.

The most promissing distinction is between concrete and abstract entities.

In [302]:
#show(Und1[16], data, highlight='Und1_phr')

##### DBR "speak"

The addressees of "speak" are not directly involved in the event. They are, however, affected because they experience the speech. In terms of volition, what is significant for this verb is not so much whether the speech benefits the addressees but that the experience requires a volitional/sentient listener.

In [323]:
#show(Und1[41], data, highlight='Und1_phr')

#### Undergoer 2

In [332]:
Und2 = evaluate('Und2')
print(f'Number of verbs to be evaluated: {len(Und2)}')

Number of verbs to be evaluated: 15


##### BDL "separate"

"Separate" can be a physical separation where an Undergoer is taken away from another Undergoer (the latter probably not affected). Or the verb can be "distinguish" which is basically a cognitive task and the Undergoers are not affected. 

In [339]:
#show(Und2[1], data, highlight='Und2_phr')

##### Locative complements

Normally, locative complements are treated as neutral because they merely describe a location of an event, or the source or the endpoint. They are not themselves affected by the event.

There are exceptions, however. When YHWH sends plague "among Israel", the participant is certainly affected:

In [350]:
#show(Und2[7], data, highlight='Und2_phr')

And also when YHWH sends fear into the hearts of the people:

In [352]:
#show(Und2[9], data, highlight='Und2_phr')

### Reviewing complement phrases

The final validation step involves a manual inspection of all complement phrases in the text not marked for [+VOL] and [+AFF].

First we import the latest update of the data:

In [11]:
#A dictionary of columns to be imported as integers.
int_cols = {col:'Int64' for col in ['clause','Act_phr','Und1_phr','Und2_phr']}

data = pd.read_csv(f'{PATH}Lev17-26.Volition_Affectedness_all_cor_2.csv', dtype=int_cols)

data.head()

,Unnamed: 0,Unnamed: 0.1,clause,lex,Act_phr,Act_vol,Act_aff,Und1_phr,Und1_vol,Und1_aff,Und2_phr,Und2_vol,Und2_aff,comment
0,0,0,439650,DBR[,688348,y,n,688349,y,y,NaN,NaN,NaN,NaN
1,1,1,439651,>MR[,688350,y,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,439652,DBR[,688351,y,n,688352,y,y,NaN,NaN,NaN,NaN
3,3,3,439653,>MR[,688354,y,n,688355,n,n,NaN,NaN,NaN,NaN
4,4,4,439655,YWH[,688360,y,n,688358,n,n,NaN,NaN,NaN,NaN


We extract all complements in the text (Lev 17-26) and check if they have been annotated:

In [12]:
all_Cmpl = '''
book book=Leviticus
 chapter chapter=17|18|19|20|21|22|23|24|25|26
  clause
   phrase function=Cmpl
'''

all_Cmpl = A.search(all_Cmpl)

  0.45s 481 results


In [13]:
anno_complements = []

for r in all_Cmpl:
    Cmpl = r[3]
    
    cols = ['Act', 'Und1', 'Und2']
    for c in cols:
        
        #Check if Cmpl in particular column:
        if Cmpl in (list(data[f'{c}_phr'])):
            
            #check if +Vol and +Aff:
            vol = data[data[f'{c}_phr'] == Cmpl][f'{c}_vol'].item()
            aff = data[data[f'{c}_phr'] == Cmpl][f'{c}_aff'].item()
            
            if vol == 'y' and aff == 'y':
                anno_complements.append(Cmpl)
                
print(f'Number of annotated complements: {len(anno_complements)}')

Number of annotated complements: 116


In [14]:
complements = []

for r in all_Cmpl:
    Cmpl = r[3]
    if Cmpl not in anno_complements:
        complements.append(Cmpl)
        
print(f'Number of complements to be annotated/reviewed: {len(complements)}')

Number of complements to be annotated/reviewed: 365


In [15]:
def showCmpl(Cmpl, data=data):
    clause = L.u(Cmpl, 'clause')[0]
    print(f'Clause: {clause}\nPhrase: {Cmpl}')
    
    cols = ['Act', 'Und1', 'Und2']
    for c in cols:
        
        #Check if Cmpl in particular column:
        if Cmpl in (list(data[f'{c}_phr'])):
            
            #Extract if Vol and Aff:
            vol = data[data[f'{c}_phr'] == Cmpl][f'{c}_vol'].item()
            aff = data[data[f'{c}_phr'] == Cmpl][f'{c}_aff'].item()
            
            print(f'Existing annotation: Vol = {vol} and Aff = {aff}')
    
    A.pretty(clause, highlights={Cmpl: 'gold'})

#### "In my name"

Is God's name affected by the swearing. On the one hand, the name is an abstract and neither volitional nor affective. On the other hand, the name represents the person bearing it and may therefore very well be affected in the sense of having the person's reputation damaged:

In [463]:
#showCmpl(complements[56])

In [331]:
n = 0

In [332]:
print(f'Case {n}')
showCmpl(complements[n])
n+=1

Case 0
Clause: 439653
Phrase: 688355
Existing annotation: Vol = n and Aff = n
